# Geoscripting Exercise 12 GISRSArtists
## 22 January 2019

Imports

In [1]:
from geopy.geocoders import Nominatim
import rasterio
from owslib.wcs import WebCoverageService
from owslib.wfs import WebFeatureService
from python.createBoundingBox import createBoundingBox
from python.writeGeotiffToFile import writeGeotiffToFile
from python.getGeotiffFromWebCoverageService import getGeotiffFromWebCoverageService
from python.getDataFromWebFeatureService import getDataFromWebFeatureService
from python.calculateBuildingVolume import calculateBuildingVolume
from python.visualizeBuildingVolume import visualizeBuildingVolume
from python.geocodePlacenameToCoordinates import geocodePlacenameToCoordinates
from python.reproject import reproject

In [5]:
# get coordinates of Wageningen and reproject to RD New
wageningen = geocodePlacenameToCoordinates('Wageningen University')
x, y = reproject(wageningen, '4326', '28992')

# create bounding box around Wageningen
bbox = createBoundingBox(x, y)

# get wcs of Wageningen
url = 'http://geodata.nationaalgeoregister.nl/ahn2/wcs?service=WCS'
version = '1.0.0'
direc = 'data'
frmt = 'GEOTIFF_FLOAT32'
getGeotiffFromWebCoverageService(url, version, bbox, 
                                 'ahn2_05m_ruw', frmt, direc, 'AHN2_05m_DSM')
getGeotiffFromWebCoverageService(url, version, bbox, 
                                 'ahn2_05m_int', frmt, direc, 'AHN2_05m_DTM')


In [7]:
DSM = rasterio.open("./data/AHN2_05m_DSM.tif", driver="GTiff")
DTM = rasterio.open("./data/AHN2_05m_DTM.tif", driver="GTiff")
buildingsHeight = DSM.read() - DTM.read()
print(type(buildingsHeight))

kwargs = DSM.meta # Copy metadata of rasterio.io.DatasetReader
writeGeotiffToFile(buildingsHeight, 'output', 'buildingsHeight', kwargs)

<class 'numpy.ndarray'>


AttributeError: module 'os' has no attribute 'mkdirs'